In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from PIL import Image
import pandas as pd

from tensorflow.keras.losses import MSE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, r2_score

import h5py
from tqdm import tqdm

from pykrige.ok3d import OrdinaryKriging3D
from pykrige.uk3d import UniversalKriging3D

import time
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import curve_fit
import numba

In [ ]:
def structure_obs(N, random_range):
    x = np.arange(0, N, 7)
    y = np.arange(0, N, 7)
    x_list, y_list = np.meshgrid(x, y)

    x_coord = np.empty(x_list.size, dtype=int)
    y_coord = np.empty(y_list.size, dtype=int)

    for i, (x_val, y_val) in enumerate(zip(x_list.flatten(), y_list.flatten())):
        if (i+1) % 64 != 0:
            x_val += rand.randint(0, random_range)
            y_val += rand.randint(0, random_range)

        x_coord[i] = x_val if x_val < N else N - 1
        y_coord[i] = y_val if y_val < N else N - 1

    return x_coord, y_coord

# 3D-Kriging

In [ ]:
true_u_vect = np.load('./Dataset_sw/true_u_full.npy', mmap_mode="r")[-3000:]
true_v_vect = np.load('./Dataset_sw/true_v_full.npy', mmap_mode="r")[-3000:]
true_h_vect = np.load('./Dataset_sw/true_h_full.npy', mmap_mode="r")[-3000:]

# Constants
N = 64
datasize = 3000  # Total number of samples to process

# Initialize arrays
position_data = np.zeros((datasize, 100, 3))  # Stores x,y coordinates for 100 points in each of the datasize samples
value_data = np.zeros((datasize, 100, 3))  # Stores u,v,h values for 100 points in each of the datasize samples
t = 0
# Data processing
for index in range(datasize):
    x_coord, y_coord = structure_obs(N, 2)  # Get 100 random points

    # Sampling points from each vector (u, v, h)
    for vect, slice_idx in zip([true_u_vect, true_v_vect, true_h_vect], range(3)):
        Zi = vect[index][y_coord, x_coord]  # Sample values at the points
        value_data[index, :, slice_idx] = Zi  # Store sampled values

    # Store the coordinates
    position_data[index, :, 0] = x_coord
    position_data[index, :, 1] = y_coord
    position_data[index, :, 2] = t
    t+=0.00001

# `value_data` already contains u,v,h values in the last dimension
vor_u_data, vor_v_data, vor_h_data = value_data[..., 0], value_data[..., 1], value_data[..., 2]

In [ ]:
def calculate_rrmse(true, predicted):
    """
    Calculate the Relative Root Mean Square Error (R-RMSE).
    """
    # Reshape the arrays for RMSE calculation
    true_reshaped = true.reshape(5, 64, 64, 1)
    predicted_reshaped = predicted.reshape(5, 64, 64, 1)

    # Calculate the Root Mean Square Error (RMSE)
    rmse = np.sqrt(np.mean((true_reshaped - predicted_reshaped) ** 2))

    # Calculate the mean of the true data
    mean_true = np.mean(true_reshaped)

    # Avoid division by zero by handling the case where the mean of the true data is zero
    if mean_true == 0:
        return np.inf

    print(mean_true)
    
    # Calculate the Relative Root Mean Square Error (R-RMSE)
    rrmse = rmse / mean_true
    return rrmse

# Prepare grid for Kriging interpolation
gridx = np.arange(0.0, 64, 1)
gridy = np.arange(0.0, 64, 1)
gridz = np.arange(0.0, 0.00010, 0.00001)

# Initialize metrics storage
ssim_vals = []
psnr_vals = []
rrmse_vals = []

# Extracting x-axis time positions for the Kriging
x_time = position_data[:5].reshape((100 * 5, 3))[:, 2]

def execute_kriging(X, Y, Z, Y_values, gridx, gridy, gridz):
    """
    Perform Kriging interpolation and return results.
    """
    ok3d = OrdinaryKriging3D(X, Y, Z, Y_values, variogram_model="spherical", nlags=20)
    k3d, ss3d = ok3d.execute("grid", gridx, gridy, gridz)
    return k3d, ss3d

# Loop over specified data range, perform Kriging, and compute SSIM & PSNR
total_time = 0
k = 0
for i in range(0, 295, 5):
    X_va_2 = position_data[i + 300 * k:(i + 5) + 300 * k].reshape((100 * 5, 3))
    y_va_2 = value_data[i + 300 * k:(i + 5) + 300 * k].reshape((100 * 5, 3))[:, 0]
    result = true_u_vect[-10 * 300:][(i + 5) + 300 * k:(i + 10) + 300 * k]

    start_time = time.time()
    # Corrected argument list: Separate Z coordinates and Y_values
    k3d1, ss3d = execute_kriging(X_va_2[:, 0], X_va_2[:, 1], X_va_2[:, 2], y_va_2, gridx, gridy, gridz)
    total_time += time.time() - start_time

    # Assuming result and k3d1 are compatible for comparison; adjust as needed
    for true, pred in zip(result, k3d1[-5:]):
        ssim_vals.append(ssim(true, pred, data_range=pred.max() - pred.min()))
        psnr_vals.append(psnr(true, pred, data_range=pred.max() - pred.min()))
        
        rrmse_val = calculate_rrmse(true, pred)
        rrmse_vals.append(rrmse_val)
        print(rrmse_val)

# Calculate and print average metrics
avg_ssim = np.mean(ssim_vals)
avg_psnr = np.mean(psnr_vals)

#RRMSE You need to test on three dim and concat get a full field, then you can cal it.
#average_rrmse = np.mean(rrmse_vals)
print(f"Average SSIM: {avg_ssim}")
print(f"Average PSNR: {avg_psnr}")
#print(f"Average R-RMSE: {average_rrmse}")

print(f"Total Inference Time: {total_time} seconds")


# 2D-Krging

In [ ]:
import random as rand
true_u_vect = np.load('./Dataset_sw/true_u_full.npy', mmap_mode="r")[-3000:]
true_v_vect = np.load('./Dataset_sw/true_v_full.npy', mmap_mode="r")[-3000:]
true_h_vect = np.load('./Dataset_sw/true_h_full.npy', mmap_mode="r")[-3000:]

# Constants
N = 64
datasize = 3000  # Total number of samples to process

# Initialize arrays
position_data = np.zeros((datasize, 100, 2))  # Stores x,y coordinates for 100 points in each of the datasize samples
value_data = np.zeros((datasize, 100, 3))  # Stores u,v,h values for 100 points in each of the datasize samples
t = 0
# Data processing
for index in range(datasize):
    x_coord, y_coord = structure_obs(N, 2)  # Get 100 random points

    # Sampling points from each vector (u, v, h)
    for vect, slice_idx in zip([true_u_vect, true_v_vect, true_h_vect], range(3)):
        Zi = vect[index][y_coord, x_coord]  # Sample values at the points
        value_data[index, :, slice_idx] = Zi  # Store sampled values

    # Store the coordinates
    position_data[index, :, 0] = x_coord
    position_data[index, :, 1] = y_coord

In [ ]:
import numpy as np
import time
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import curve_fit
from pykrige.ok3d import OrdinaryKriging3D
import numba

# Assuming you have a working OrdinaryKriging3D implementation available
def calculate_rrmse(true, predicted):
    """
    Calculate the Relative Root Mean Square Error (R-RMSE).
    """
    # Reshape the arrays for RMSE calculation
    true_reshaped = true.reshape(5, 64, 64, 1)
    predicted_reshaped = predicted.reshape(5, 64, 64, 1)

    # Calculate the Root Mean Square Error (RMSE)
    rmse = np.sqrt(np.mean((true_reshaped - predicted_reshaped) ** 2))

    # Calculate the mean of the true data
    mean_true = np.mean(true_reshaped)

    # Avoid division by zero by handling the case where the mean of the true data is zero
    if mean_true == 0:
        return np.inf

    print(mean_true)
    
    # Calculate the Relative Root Mean Square Error (R-RMSE)
    rrmse = rmse / mean_true
    return rrmse

# Prepare grid for Kriging interpolation
gridx = np.arange(0.0, 64, 1)
gridy = np.arange(0.0, 64, 1)
gridz = np.arange(0.0, 0.00010, 0.00001)

# Initialize metrics storage
ssim_vals = []
psnr_vals = []
rrmse_vals = []

# Extracting x-axis time positions for the Kriging
x_time = position_data[:5].reshape((100 * 5, 3))[:, 2]

def execute_kriging(X, Y, Z, Y_values, gridx, gridy, gridz):
    """
    Perform Kriging interpolation and return results.
    """
    ok3d = OrdinaryKriging3D(X, Y, Z, Y_values, variogram_model="spherical", nlags=20)
    k3d, ss3d = ok3d.execute("grid", gridx, gridy, gridz)
    return k3d, ss3d

# Loop over specified data range, perform Kriging, and compute SSIM & PSNR
total_time = 0
k = 0
for i in range(0, 295, 5):
    X_va_2 = position_data[i + 300 * k:(i + 5) + 300 * k].reshape((100 * 5, 3))
    y_va_2 = value_data[i + 300 * k:(i + 5) + 300 * k].reshape((100 * 5, 3))[:, 0]
    result = true_u_vect[-10 * 300:][(i + 5) + 300 * k:(i + 10) + 300 * k]

    start_time = time.time()
    # Corrected argument list: Separate Z coordinates and Y_values
    k3d1, ss3d = execute_kriging(X_va_2[:, 0], X_va_2[:, 1], X_va_2[:, 2], y_va_2, gridx, gridy, gridz)
    total_time += time.time() - start_time

    # Assuming result and k3d1 are compatible for comparison; adjust as needed
    for true, pred in zip(result, k3d1[-5:]):
        ssim_vals.append(ssim(true, pred, data_range=pred.max() - pred.min()))
        psnr_vals.append(psnr(true, pred, data_range=pred.max() - pred.min()))
        
        rrmse_val = calculate_rrmse(true, pred)
        rrmse_vals.append(rrmse_val)
        print(rrmse_val)

# Calculate and print average metrics
avg_ssim = np.mean(ssim_vals)
avg_psnr = np.mean(psnr_vals)
average_rrmse = np.mean(rrmse_vals)
print(f"Average SSIM: {avg_ssim}")
print(f"Average PSNR: {avg_psnr}")
print(f"Average R-RMSE: {average_rrmse}")

print(f"Total Inference Time: {total_time} seconds")


In [ ]:
from sklearn.linear_model import LinearRegression

true_set = []
predict_set = []
import time
total_time = 0
for k in range(0, 300,5):
    i = 2
    # Sample usage
    img_num_0 = k
    img_num_1 = k + 1
    img_num_2 = k + 2
    img_num_3 = k + 3
    img_num_4 = k + 4
    img_num_5 = k + 5
    img_num_6 = k + 6

    img_num_7 = k + 7

    img_num_8 = k + 8
    img_num_9 = k + 9
    img_num_10 = k + 10
    gridx = np.arange(0, 64, 1, dtype='float64')
    gridy = np.arange(0, 64, 1, dtype='float64')

    zstar_0 = np.flipud(kriging_interpolation(position_data[img_num_0], value_data[img_num_0,:,i],  gridx, gridy))
    zstar_1 = np.flipud(kriging_interpolation(position_data[img_num_1], value_data[img_num_1,:,i],  gridx, gridy))
    zstar_2 = np.flipud(kriging_interpolation(position_data[img_num_2], value_data[img_num_2,:,i],  gridx, gridy))
    zstar_3 = np.flipud(kriging_interpolation(position_data[img_num_3], value_data[img_num_3,:,i],  gridx, gridy))
    zstar_4 = np.flipud(kriging_interpolation(position_data[img_num_4], value_data[img_num_4,:,i],  gridx, gridy))
    zstar_5 = np.flipud(kriging_interpolation(position_data[img_num_5], value_data[img_num_5,:,i],  gridx, gridy))


    # Flatten images into 1D vectors
    X = np.array([zstar_0.flatten(), zstar_1.flatten(), zstar_2.flatten(), zstar_3.flatten(), zstar_4.flatten()]).T

    # Target value is the next image zstar_5
    y = zstar_5.flatten()

    # Create linear regression model
    model = LinearRegression()

    # Fit the model
    model.fit(X, y)

    start_time = time.time()

    # Perform prediction for the next five steps
    next_image_1 = model.predict(np.array([zstar_0.flatten(), zstar_1.flatten(), zstar_2.flatten(), zstar_3.flatten(), zstar_4.flatten()]).T)
    next_image_2 = model.predict(np.array([zstar_1.flatten(), zstar_2.flatten(), zstar_3.flatten(), zstar_4.flatten(), next_image_1]).T)
    next_image_3 = model.predict(np.array([zstar_2.flatten(), zstar_3.flatten(), zstar_4.flatten(), next_image_1, next_image_2]).T)
    next_image_4 = model.predict(np.array([zstar_3.flatten(), zstar_4.flatten(), next_image_1, next_image_2, next_image_3]).T)
    next_image_5 = model.predict(np.array([zstar_4.flatten(), next_image_1, next_image_2, next_image_3, next_image_4]).T)

    # End time for the current iteration
    end_time = time.time()

    # Calculate the time taken for the current iteration
    iteration_time = end_time - start_time
    print("Total time taken: ", iteration_time, "seconds")

    # Accumulate the time for all iterations
    total_time += iteration_time

    # Append to true_set and predict_set
    true_pic_1 = true_v_vect[img_num_5]
    true_pic_2 = true_v_vect[img_num_6]
    true_pic_3 = true_v_vect[img_num_7]
    true_pic_4 = true_v_vect[img_num_8]
    true_pic_5 = true_v_vect[img_num_9]

    true_set.append([true_pic_1, true_pic_2, true_pic_3, true_pic_4, true_pic_5])
    predict_set.append([next_image_1, next_image_2, next_image_3, next_image_4, next_image_5])

true_set = np.array(true_set)
predict_set = np.array(predict_set)

In [ ]:
predict_set = predict_set.reshape((300,64,64,1)).astype(predict_set.dtype)

true_set = true_set.reshape((300,64,64,1))

ssim_1 = ssim(true_set, predict_set, multichannel=True)

psnr_1 = psnr(true_set, predict_set, data_range=predict_set.max() - predict_set.min())
# Same as 3D-Kriging, then cal it
#rrmse = calculate_rrmse(true_set,predict_set)